## Data preprocessing



In [6]:
import numpy as np
import pandas as pd
from keras.regularizers import l1
from PIL import Image
from tensorflow.keras import optimizers
import os
from sklearn.model_selection import train_test_split
import keras
from PIL import Image
from tensorflow.keras.layers import Input,Conv2D, MaxPooling2D, UpSampling2D,concatenate,BatchNormalization,Conv2DTranspose,Flatten
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model,load_model
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from keras import layers
import shutil
from sklearn import preprocessing
from sklearn.cluster import KMeans,Birch,MeanShift
from sklearn.mixture import GaussianMixture
from random import choice
import tensorflow as tf
import imageio
import math
from google.colab import drive


In [7]:
# List all files in the '/content/' directory
a = os.listdir('/content/')
# Loop through the files and attempt to remove directories that have integer names
for i in a:
    try:
        # If the file name is an integer, delete the directory
        int(i)
        shutil.rmtree('/content/' + i)
    except:
        # Print the file name if it's not an integer or can't be deleted
        print(i)

# Install the Kaggle library to access datasets
!pip install kaggle

# Mount Google Drive to access files
drive.mount('/content/drive')

# Create the .kaggle directory to store API key for Kaggle access
!mkdir ~/.kaggle

# Copy the kaggle.json file (API key) from your Google Drive to the .kaggle directory
!cp /content/drive/MyDrive/brain/kaggle.json ~/.kaggle/kaggle.json

# Set permissions for the kaggle.json file to make it readable
!chmod 600 ~/.kaggle/kaggle.json

# Download the brain dataset from Kaggle
!kaggle datasets download -d othmanebazine/braindata

# Extract the downloaded dataset zip file
zip = zipfile.ZipFile('braindata.zip')
zip.extractall()

# Rename the extracted 'data' folder to 'dataset'
os.rename('/content/data', '/content/dataset')

# Load the brain data CSV file from Google Drive into a pandas DataFrame
df_filled = pd.read_csv('/content/drive/MyDrive/brain/oasis_cross-sectional.csv')

# Load test and train index data from text files
index_test = np.loadtxt('test_index.txt', dtype=str).astype('object')
index_train = np.loadtxt('train_index.txt', dtype=str).astype('object')


.config
sample_data


In [16]:
import numpy as np
from sklearn.utils import shuffle

# Function to create training data by pairing images based on label and class
def sub_create_data(image, label, classe, state):
    # Initialize lists to store image pairs and corresponding labels
    x_train = []
    label_train = []
    class_train = []

    # Loop through each unique label in the dataset
    for i in np.unique(label):
        # Depending on the state, find indices where the class matches the condition
        if state == 1:
            index_1 = np.where((label == i) & (classe == 1))[0]
            index_0 = np.where((label == i) & (classe == 0))[0]
        elif state == 0:
            index_1 = np.where((label == i) & (classe == 0))[0]
            index_0 = np.where((label == i) & (classe == 1))[0]
        elif state == 2:
            index_1 = np.where((label == i) & (classe == 1))[0]
            index_0 = np.where((label == i) & (classe == 2))[0]

        # Create pairs of images based on index_1 and index_0
        for j in index_1:
            for l in index_0:
                img2 = image[j]  # Image with index j
                img3 = image[l]  # Image with index l

                # Combine the two images into one array and add to the training set
                s = np.array([img2, img3])
                x_train.append(s)
                label_train.append(i)

    # Convert lists to NumPy arrays for better performance
    x_train = np.array(x_train).astype('uint8')
    label_train = np.array(label_train).astype('int16')

    # Shuffle the data before returning
    x_train, label_train = shuffle(x_train, label_train)

    return x_train, label_train

# Function to create the final dataset by generating triplets of images for training
def create_data(image, label, classe, dimentia, state=0, nmbr=50):
    # Call sub_create_data to get paired images and labels
    x_train, label_train = sub_create_data(image, label, classe, state)

    # Initialize lists to store the triplets and corresponding labels
    X_train = []
    Y_train = []
    Label = []

    # Number of unique labels and a counter
    k = len(np.unique(label_train))
    l = 0
    shape = (224, 192, 1)  # Shape of the images (height, width, channels)

    # Loop through each unique label in the dataset
    for i in np.unique(label_train):
        # Get the indices where label_train equals the current label
        index0 = np.where(label_train == i)[0]
        lenght = len(index0)

        # Generate triplets of images for training
        for _ in range(nmbr):
            t = np.unique(label_train)[l % k]  # Select a different label
            if t != i:
                # Find the indices for the selected different label
                index1 = np.where(label_train == t)[0]

                # Choose images from the current label and the different label
                choice0 = index0[l % lenght]
                choice1 = np.random.choice(index1)

                # Create a triplet by concatenating the images along the channel axis
                s = np.concatenate([
                    x_train[choice0][0].reshape(shape),  # First image from the pair
                    x_train[choice0][1].reshape(shape),  # Second image from the pair
                    x_train[choice1][0].reshape(shape)   # Third image from different label
                ], axis=2)

                # Add the triplet and the corresponding label to the training set
                X_train.append(s)
                Y_train.append(x_train[choice1][1].reshape(shape))  # Second image of the different label
                Label.append(label_train[choice1])  # Label of the different image

            l += 1

    # Convert lists to NumPy arrays
    X_train = np.array(X_train).astype('uint8')
    Y_train = np.array(Y_train)
    Label = np.array(Label)

    # Shuffle the data before returning
    X_train, Y_train, Label = shuffle(X_train, Y_train, Label)

    return X_train, Y_train, Label

# Call the create_data function to generate the training data
x_train, y_train, Label = create_data(images_train, labels_train, classes_train, state=1)


## model

In [ ]:
# Define the input shape for the autoencoder model
input_img = Input(shape=(224, 192, 3))  # Input shape with 3 channels (change if needed)

# Split input image into two parts: X (first 2 channels) and Y (third channel)
X = input_img[:, :, :, :2]  # Select the first 2 channels from the input
Y = input_img[:, :, :, 2]   # Select the third channel from the input

# Reshape the input to match the desired format for the model
X = layers.Reshape((224, 192, 2))(X)
Y = layers.Reshape((224, 192, 1))(Y)

# ------------------ Encoder 1 (for X input) ------------------

# First layer of convolutions followed by pooling
conv1 = Conv2D(8, (3, 3), activation='relu', padding='same')(X)
conv1 = Conv2D(8, (3, 3), activation='relu', padding='same')(conv1)
conv1 = Conv2D(8, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D((2, 2), padding='same')(conv1)

# Second block of convolutions followed by pooling
conv2 = Conv2D(16, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv2)
conv2 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D((2, 2), padding='same')(conv2)

# Third block of convolutions followed by pooling
conv3 = Conv2D(32, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv3)
conv3 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D((2, 2), padding='same')(conv3)

# Fourth block of convolutions followed by pooling
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool3)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D((2, 2), padding='same')(conv4)

# Final convolution layer for the encoder
conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv5)

# Store the pooling layers for later concatenation in the decoder
pools = [pool1, pool2, pool3, pool4]

# ------------------ Encoder 2 (for Y input) ------------------

# First layer of convolutions for Y followed by pooling
Y_2 = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(Y)
Y_2 = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(Y_2)
Y_2 = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(Y_2)
Y_2 = MaxPooling2D((2, 2), padding='same')(Y_2)

# Second block of convolutions followed by pooling
Y_2 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(Y_2)
Y_2 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(Y_2)
Y_2 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(Y_2)
Y_2 = MaxPooling2D((2, 2), padding='same')(Y_2)

# Third block of convolutions followed by pooling
Y_2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(Y_2)
Y_2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(Y_2)
Y_2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(Y_2)
Y_2 = MaxPooling2D((2, 2), padding='same')(Y_2)

# Fourth block of convolutions followed by pooling
Y_2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(Y_2)
Y_2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(Y_2)
Y_2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(Y_2)
Y_2 = MaxPooling2D((2, 2), padding='same')(Y_2)

# ------------------ Merge Encoders ------------------

# Concatenate the outputs from Encoder 1 and Encoder 2
a = concatenate([conv5, Y_2], axis=-1)

# ------------------ Decoder ------------------

# First layer of the decoder
conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(a)
conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)
conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)

# First upsampling and concatenation
concat1 = concatenate([conv6, pools[3]], axis=-1)
up1 = UpSampling2D((2, 2))(concat1)

# Second layer of the decoder
conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up1)
conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)
conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)

# Second upsampling and concatenation
concat2 = concatenate([conv7, pools[2]], axis=-1)
up2 = UpSampling2D((2, 2))(concat2)

# Third layer of the decoder
conv8 = Conv2D(16, (3, 3), activation='relu', padding='same')(up2)
conv8 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv8)
conv8 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv8)

# Third upsampling and concatenation
concat3 = concatenate([conv8, pools[1]], axis=-1)
up3 = UpSampling2D((2, 2))(concat3)

# Fourth layer of the decoder
conv9 = Conv2D(8, (3, 3), activation='relu', padding='same')(up3)
conv9 = Conv2D(8, (3, 3), activation='relu', padding='same')(conv9)
conv9 = Conv2D(8, (3, 3), activation='relu', padding='same')(conv9)

# Fourth upsampling and concatenation
concat4 = concatenate([conv9, pools[0]], axis=-1)
up4 = UpSampling2D((2, 2))(concat4)

# Final output layer to generate the decoded image
decoded = Conv2D(4, (3, 3), activation='relu', padding='same')(up4)
decoded = Conv2D(1, (3, 3), activation='relu', padding='same')(decoded)

# ------------------ Autoencoder Model ------------------

# Create the autoencoder model that maps input_img to decoded output
autoencoder = Model(input_img, decoded)

# Compile the model using Adam optimizer and Mean Absolute Error (MAE) loss
autoencoder.compile(optimizer='adam', loss='mae')

# Display the model summary
autoencoder.summary()


In [ ]:
autoencoder.fit(x_train,y_train,epochs=50,batch_size=128,shuffle=True,validation_split=0.15)


In [ ]:
autoencoder.save("/content/drive/MyDrive/brain/denoise_decoder1.2.1.h5")